# Threshold Spreading

In [56]:
# Imports

import numpy as np
import networkx as nx

In [57]:
G = nx.read_edgelist('M1/edges.csv', delimiter = ',')
print(G)

Graph with 1429 nodes and 19357 edges


We first create a function which computes the new infected nodes using threshold reinforcement:

In [85]:
# Modified SI function using threshold
def new_infected_threshold(G, I, kappa, t):
    '''SI function which uses thresholds to determine which new 
    nodes are infected by their neighbours in one step of the infection process.
    
    Args:
        G (nx.Graph): Graph to perform algorithm on
        I (set): Set of already infected nodes
        kappa (int): Threshold
        t (int): Time step of current infection step

    Return:
        new_infected (set): Set of newly infected nodes.
    '''

    # Create a new set of the newly infected nodes
    new_infected = set()

    # Check if time step t is 1 and set threshold to 1 if true
    if t == 1:
        kappa = 1

    # Iterate over all nodes in the network which are not already infected
    for u in set(G.nodes) - I:

        # # Print for debugging
        # print(f"Node {u}: Neighbors: {set(G.neighbors(u))}, Infected: {I}")

        # Check if the amount of infected neighbors passes the threshold
        if len(set(G.neighbors(u)) & I) >= kappa:
            new_infected.add(u)

    return new_infected

We now create a function to run a complete infection process on the network:

In [88]:
def threshold_infection_process(G, s, kappa):
    '''Perform SI infection process on a complete network using threshold reinforcement.
    
    Args:
        G (nx.Graph): Graph to perform algorithm on
        s (int): Starting node for the infection process
        kappa (int): Threshold

    Return:
        t (int): Time step reached when spreading stopped
        all_infected (bool): Whether all nodes have been infected
    '''

    I = {s} # Set of infected nodes, starts with only staring node infected
    t = 0 # Initialize time step

    # Time step loop, continues until all nodes are infected
    while len(I) < len(G.nodes):

        # Increment t
        t += 1

        # Find new infected nodes
        new_infected = new_infected_threshold(G, I, kappa, t)

        # # Print for debugging
        # print(f"Time step {t}: Newly infected nodes: {new_infected}")

        # Break if no new nodes are infected
        if len(new_infected) == 0:
            break

        # Add new infected nodes to I
        I = I.union(new_infected)

        # # Print for debugging
        # print(f"Time step {t}: Infected nodes: {I}")

    # Keep bool of whether all nodes have been infected
    all_infected = (len(I) == len(G.nodes))

    return t, all_infected

We now check the infection of different values of kappa:

In [108]:
# Keep list of kappa threshold values
thresholds = list(range(1, 6))

# Iterate over each threshold value
for kappa in thresholds:
    steps_used_successfull = [] # List of steps used for each starting node which infected whole G
    steps_used_failure = [] # List of steps used for each starting node which did not infected whole G
    success_rate = .0 # Ratio of successfull infections
    V = len(G.nodes) # Amount of nodes

    # Iterate over all possible starting nodes
    for s in G.nodes:
        final_step, success = threshold_infection_process(G, s, kappa)

        # Append the amount of steps used to list depending on whether total infection was successfull
        if success:
            steps_used_successfull.append(final_step)
        else:
            steps_used_failure.append(final_step)

        # Increment success_rate
        success_rate += success / V

    # Compute average steps statistics
    average_steps_total = sum(steps_used_successfull + steps_used_failure)/len(steps_used_successfull + steps_used_failure)
    if len(steps_used_successfull) == 0:
        average_steps_successfull = 0
    else:
        average_steps_successfull = sum(steps_used_successfull)/len(steps_used_successfull)
    if len(steps_used_failure) == 0:
        average_steps_failure = 0
    else:
        average_steps_failure = sum(steps_used_failure)/len(steps_used_failure)

    print(f'Kappa {kappa}:')
    print(f'Success rate: {success_rate:.4f}')
    print('Average steps taken:')
    print(f'Total: {average_steps_total:.4f}, Successfull: {average_steps_successfull:.4f}, Failure: {average_steps_failure:.4f}, \n')

Kappa 1:
Success rate: 1.0000
Average steps taken:
Total: 5.1896, Successfull: 5.1896, Failure: 0.0000, 

Kappa 2:
Success rate: 0.9993
Average steps taken:
Total: 6.9405, Successfull: 6.9440, Failure: 2.0000, 

Kappa 3:
Success rate: 0.0000
Average steps taken:
Total: 10.5745, Successfull: 0.0000, Failure: 10.5745, 

Kappa 4:
Success rate: 0.0000
Average steps taken:
Total: 12.9741, Successfull: 0.0000, Failure: 12.9741, 

Kappa 5:
Success rate: 0.0000
Average steps taken:
Total: 11.2883, Successfull: 0.0000, Failure: 11.2883, 



KeyboardInterrupt: 